In [ ]:
!pip install datatable

In [ ]:
import ast
import re
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import datatable as dt
from datatable import f,count,by,sort,ifelse,update
from google.colab import data_table
data_table.enable_dataframe_formatter()
dt.options.display.head_nrows=5
dt.options.display.use_colors=True

In [ ]:
!cp /content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/SQL_datos/messages_classified_tidy_version_1.csv .

In [ ]:
messages_sc_dt = dt.fread('/content/messages_classified_tidy_version_1.csv',
                          columns=['col_num','message_id','message_preview','message_flag', 'has_URL', 'has_EMAIL_CTA', 'URL_Domain', 'url_tidy'],na_strings=["","NaN"])

In [ ]:
del messages_sc_dt['col_num']

In [ ]:
message_nan_counts_df = messages_sc_dt.countna().to_pandas()

In [ ]:
def my_message_preview_tidy(message):
    rgx_pat = r'\:[\d]{2,3}|\b[\w]{2}\b|\t'
    if message is not None:
        return ast.literal_eval(re.sub(rgx_pat,"",str(message)))
    else:
        return None

In [ ]:
def my_custom_string_length(txt):
    return [  (len(re.findall(r'\w+', str(msg))),len(msg)) if msg is not None  else None for msg in txt ]

In [ ]:
def my_url_list_len(lista):
    if lista is not None:
        return len(ast.literal_eval(lista))
    else:
        return None

In [ ]:
message_preview_tidy_dt = dt.Frame({'message_preview':my_message_preview_tidy(
    list(messages_sc_dt[:,f.message_preview
                          ].to_dict().values())[0])
})

In [ ]:
messges_words_length_dt = dt.Frame({'no_of_words_length':my_custom_string_length(
    list(message_preview_tidy_dt[:,f.message_preview].to_dict().values())[0])
},types=['obj64'])

In [ ]:
messages_no_of_urls_dt = dt.Frame({'no_of_urls':[my_url_list_len(item) for item in list(messages_sc_dt[:,f.url_tidy].to_dict().values())[0]]})

In [ ]:
msg_word_len_dict = {'no_of_words':[],
                     'msg_length':[]
}

for item in messges_words_length_dt[:,f.no_of_words_length].to_dict().values():
    for val in item:
        if val is not None:
            w,l=val
            msg_word_len_dict['no_of_words'].append(w)
            msg_word_len_dict['msg_length'].append(l)
        else:
            msg_word_len_dict['no_of_words'].append(None)
            msg_word_len_dict['msg_length'].append(None)

In [ ]:
messages_word_len_dt = dt.Frame(msg_word_len_dict)

In [ ]:
messages_tidy_1_dt = dt.cbind(message_preview_tidy_dt,messges_words_length_dt,messages_no_of_urls_dt,messages_word_len_dt)

In [ ]:
messages_tidy_final_dt =dt.cbind(messages_sc_dt[:,[f.message_id,f.message_flag,f.has_URL,f.has_EMAIL_CTA,f.url_tidy]],
                                 messages_tidy_1_dt[:,f[:].remove(f.no_of_words_length)]
                                 )[:,[f.message_id,f.message_preview,f.message_flag,f.has_URL,f.has_EMAIL_CTA,f.no_of_words,f.msg_length,f.no_of_urls,f.url_tidy]]

In [ ]:
messages_tidy_final_dt

In [ ]:
messages_tidy_final_dt[:,update(message_flag=ifelse(f.message_flag=="spam",1,
                                                    f.message_flag=="clean",0,
                                                    f.message_flag))]

In [ ]:
messages_tidy_final_dt[:,count(),by(f.message_flag)]

In [ ]:
messages_short_dt = messages_tidy_final_dt[ ( ( (f.no_of_words==1) | (f.no_of_words==2) | (f.no_of_words==3)) & (f.has_URL==False) ),:]

In [ ]:
messages_short_dt[:,f.message_preview,f.message_flag
                  ][~dt.re.match(f.message_preview,r'(digitextracted)?([\w\W]+)?digitextracted'),:]

In [ ]:
messages_tidy_final_dt["bin_category"] = dt.qcut(messages_tidy_final_dt[:,f.no_of_words],nquantiles=20)

In [ ]:
messages_tidy_final_dt[:,count(),by(f.bin_category)]

In [ ]:
messages_tidy_final_dt[~dt.isna(f.no_of_words),:
                       ][:,{'min':dt.min(f.no_of_words),'max':dt.max(f.no_of_words),'total':count()},by(f.bin_category)
                       ]

In [ ]:
messages_tidy_final_dt[f.no_of_words==1,:
                       ][:,count(),by(f.has_URL)]

In [ ]:
messages_tidy_final_dt[((f.has_URL==1) & (f.no_of_urls==1) & (f.message_flag=="spam")),:][:,count(),by(f.url_tidy)]

In [ ]:
messages_tidy_final_dt[~((f.no_of_words==1) & (f.has_URL==1)),:
                       ][(f.bin_category==1),:]

In [ ]:
messages_tidy_final_dt[~((f.no_of_words==1) & (f.has_URL==1)),:
                       ][((f.bin_category==0) | (f.bin_category==1) ) ,:][(f.no_of_words>6),:
                                                                          ][:,[f.message_id,f.message_preview,f.message_flag]].to_csv('my_final_messages_dt.csv')